In [1]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""

USAGE:
    Before running the sample:
    Set these environment variables with your own values:
    1) AZURE_AISTUDIO_PROJECT_CONN_STRING - The project connection string, as found in the overview page of your
       Azure AI Foundry project.
    2) AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in 
       the "Models + endpoints" tab in your Azure AI Foundry project.
=
"""
import os
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import CodeInterpreterTool
from azure.ai.agents.models import FunctionTool, ToolSet
from azure.identity import DefaultAzureCredential
from typing import Any, Callable, Set, Dict, List, Optional
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
from azure.identity import DefaultAzureCredential

load_dotenv()
AIFOUNDRY_AGENT_PROJECT_NAME = os.getenv("AIFOUNDRY_AGENT_PROJECT_NAME")
AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME")

project_client = AIProjectClient(
    endpoint=AIFOUNDRY_AGENT_PROJECT_NAME,
    credential=DefaultAzureCredential(),
)

try:
    credential = DefaultAzureCredential()
    token = credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    print(ex)

In [2]:
#Define a function to be called by the agent as a custom tool
import json

def recent_snowfall(location: str) -> str:
    """
    Fetches recent snowfall totals for a given location.
    :param location: The city name.
    :return: Snowfall details as a JSON string.
    """
    mock_snow_data = {"Seattle": "0 inches", "Denver": "2 inches"}
    snow = mock_snow_data.get(location, "Data not available.")
    return json.dumps({"location": location, "snowfall": snow})

user_functions: Set[Callable[..., Any]] = {
    recent_snowfall,
}

In [3]:
agents_client = project_client.agents

functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
agents_client.enable_auto_function_calls(toolset)

# Create your agent with the toolset
agent = agents_client.create_agent(
    model=AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME,
    name="snowfall-agent",
    instructions="You are a weather assistant tracking snowfall. Use the provided functions to answer questions.",
    toolset=toolset
)

print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_zDurhQaTAIG2rlTgt3zTE5XD


In [4]:
# Create thread for communication
def call_agent(agents_client, question):
    thread = agents_client.threads.create()
    print(f"Created thread, thread ID: {thread.id}")
    
    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=question
    )
    print(f"Created message, ID: {message.id}")

    run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")


        # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")

    agents_client.delete_agent(agent.id)
    print("Deleted agent")

In [5]:
# Send a prompt to the agent
user_prompt = "What is the recent snowfall in Seattle?"
call_agent(agents_client, user_prompt)

Created thread, thread ID: thread_kFjDYBZ1uSSps6wm1bxO8enz
Created message, ID: msg_CR02ZHujmH7y0dJgZdCDasKV
Run finished with status: completed
assistant: The recent snowfall in Seattle is 0 inches. There has been no recent snowfall reported.
user: What is the recent snowfall in Seattle?
Deleted agent
